In [1]:
import pandas as pd
import numpy as np

**read_data_small** is the function to read in the small dataset about 30 MB

In [2]:
def read_data_small():
    X_train = pd.read_csv("data_small/X_train_small.csv")
    X_test = pd.read_csv("data_small/X_test_small.csv")
    y_train = np.asarray(pd.read_csv("data_small/y_train_small.csv", header=None)[0])
    # y_train_masked = np.asarray(pd.read_csv("data_small/y_masked_train_small.csv", header=None)[0])
    return X_train, X_test, y_train

**read_data_big** is the function to read in the big dataset about 100 MB

In [3]:
def read_data_big():
    X_train = pd.read_csv("data_big/X_train_big.csv")
    X_test = pd.read_csv("data_big/X_test_big.csv")
    y_train = np.asarray(pd.read_csv("data_big/y_train_big.csv", header=None)[0])
    # y_train_masked = np.asarray(pd.read_csv("data_big/y_masked_train_big.csv", header=None)[0])
    return X_train, X_test, y_train

**read_data** is the function to read in the whole dataset about 1.5 G

In [4]:
def read_data():
    X_train = pd.read_csv("data/X_train.csv")
    X_test = pd.read_csv("data/X_test.csv")
    y_train = np.asarray(pd.read_csv("data/y_train.csv", header=None)[0])
    # y_train_masked = np.asarray(pd.read_csv("data/y_masked_train.csv", header=None)[0])
    return X_train, X_test, y_train

# Insert Your Code Here

**detect_spoofying** is the function for training the classifier and classify the results. 

Here we provide an simple example.

In [5]:
### import libraries here ###
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import scale

### code classifier here ###
def format_data(df):
    # append numberical columns
    rst = df.loc[:,["price","volume","bestBid","bestAsk",'bestBidVolume',
                    'bestAskVolume','lv2Bid', 'lv2BidVolume','lv2Ask', 
                    'lv2AskVolume', 'lv3Bid', 'lv3BidVolume', 'lv3Ask',
                    'lv3AskVolume']]
    # encode the binaries
    rst["isBid"] = df.isBid*1
    rst["isBuyer"] = df.isBuyer*1
    rst["isAggressor"] = df.isAggressor*1
    rst["type"] = (df.type == "ORDER")*1
    rst["source"] = (df.source=="USER")*1
    # parse the order id data
    rst["orderId"] = df.orderId.str.split('-').str[-1]
    rst["tradeId"] = df.tradeId.str.split('-').str[-1]
    rst["bidOrderId"] = df.bidOrderId.str.split('-').str[-1]
    rst["askOrderId"] = df.askOrderId.str.split('-').str[-1]
    # encode the multiple lable data
    tmp_operation = pd.DataFrame(pd.get_dummies(df.operation), columns=df.operation.unique()[:-1])
    rst = pd.concat([rst, tmp_operation], axis=1)
    tmp_endUserRef = pd.DataFrame(pd.get_dummies(df.endUserRef), columns=df.endUserRef.unique()[:-1])
    rst = pd.concat([rst, tmp_endUserRef], axis=1)
    return rst

def detect_spoofying(X_train, X_test, y_train):
    # clean up the data
    X_clean = format_data(pd.concat([X_train, X_test]))
    X_clean = X_clean.fillna(-1)
    X_train_clean = X_clean.iloc[:X_train.shape[0],:]
    X_test_clean = X_clean.iloc[X_train.shape[0]:,:]
    X_train_clean_scaled = scale(X_train_clean)
    X_test_clean_scaled = scale(X_test_clean)

    # fit classifier
    clf = LogisticRegression(random_state=0, class_weight='balanced').fit(X_train_clean_scaled, y_train)
    y_train_prob_pred = clf.predict_proba(X_train_clean_scaled)
    y_test_prob_pred = clf.predict_proba(X_test_clean_scaled)
    return y_train_prob_pred, y_test_prob_pred

**score** is the function that we use to compare the results. An example is provided with scoring the predictions for the training dataset. True labels for the testing data set will be supplied to score the predictions for testing dataset.

Score is based on cohen's kappa measurement. https://en.wikipedia.org/wiki/Cohen%27s_kappa

In [6]:
from sklearn.metrics import cohen_kappa_score

def score(y_pred, y_true):
    """
    y_pred: a numpy 4d array of probabilities of point assigned to each label
    y_true: a numpy array of true labels
    """
    y_pred_label = np.argmax(y_pred, axis=1)
    return cohen_kappa_score(y_pred_label, y_true)

**wrapper** is the main function to read in unzipped data and output a score for evaluation. In addition, the function returns the y probability matrix (both train and test) for grading. More details about submitting format are outlined below.

In [7]:
def wrapper():
    # read in data
    X_train, X_test, y_train = read_data_small()
    # or if you have the computational power to work with the big data set, 
    # you can comment out the read_data_samll line and uncomment the following read_data_big
    # X_train, X_test, y_train = read_data_big()
    
    # process the data, train classifier and output probability matrix
    y_train_prob_pred, y_test_prob_pred = detect_spoofying(X_train, X_test, y_train)
    
    # score the predictions
    score_train = score(y_train_prob_pred, y_train)
    # score_test = score(y_test_prob_pred, y_test)
    
    # return the scores
    return score_train, y_train_prob_pred, y_test_prob_pred

Call function wrapper:

In [8]:
score_train, y_train_prob_pred, y_test_prob_pred = wrapper()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: DataConversionWarning: Data with input dtype uint8, int64, float64, object were all converted to float64 by the scale function.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: DataConversionWarning: Data with input dtype uint8, int64, float64, object were all converted to float64 by the scale function.
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Score for training data set is:

In [9]:
score_train

0.6958535351146433

### Submission Format

The classifier function wrote should return a 4d nparray with 4 columns. The columns are corresponding to the class labels: 0, 1, 2, 3. Please see examples below.

In [10]:
y_train_prob_pred

array([[9.86452550e-001, 1.35474497e-002, 1.94722749e-118,
        9.12250926e-307],
       [9.99968643e-001, 3.13566443e-005, 2.96214030e-122,
        0.00000000e+000],
       [9.99992854e-001, 7.14648023e-006, 3.14506128e-122,
        0.00000000e+000],
       ...,
       [9.94702613e-001, 8.00320444e-006, 1.00484485e-003,
        4.28453887e-003],
       [9.99998561e-001, 1.11829215e-008, 1.73867278e-007,
        1.25416072e-006],
       [9.99999503e-001, 2.91510729e-008, 8.13796837e-008,
        3.86561393e-007]])

In [11]:
y_test_prob_pred

array([[9.99999930e-01, 7.15749322e-09, 5.88697014e-09, 5.68364629e-08],
       [9.99999829e-01, 8.01701062e-09, 2.65836581e-08, 1.36410352e-07],
       [9.99999645e-01, 2.96205392e-07, 5.34109065e-08, 5.63593627e-09],
       ...,
       [9.99889576e-01, 2.84962125e-07, 2.19083895e-05, 8.82304369e-05],
       [9.99936328e-01, 3.13940023e-08, 1.65127923e-05, 4.71282129e-05],
       [9.99972531e-01, 1.38112455e-07, 1.09861823e-05, 1.63448918e-05]])

### Write test results to csv files

Please rename your file to indicate which data set you are working with. 

- If you are using the small dataset: *y_train_prob_pred_small.csv* and *y_test_prob_pred_small.csv*
- If you are using the small dataset: *y_train_prob_pred_big.csv* and *y_test_prob_pred_big.csv*
- If you are using the original dataset: *y_train_prob_pred.csv* and *y_test_prob_pred.csv*

In [12]:
pd.DataFrame(y_train_prob_pred).to_csv("y_train_prob_pred.csv")
pd.DataFrame(y_test_prob_pred).to_csv("y_test_prob_pred.csv")